# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [21]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [22]:
new_vacay=pd.read_csv("../output_data/weather.csv")
df=pd.DataFrame(new_vacay)
df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Rikitea,-23.12,-134.97,294.37,70,41,9.90,PF,1597071860
1,Atuona,-9.80,-139.03,298.99,67,7,10.35,PF,1597071860
2,Bluff,-46.60,168.33,281.48,91,97,5.81,NZ,1597071860
3,Vardø,70.37,31.11,285.15,76,75,7.70,NO,1597071784
4,Nanortalik,60.14,-45.24,279.57,76,2,6.61,GL,1597071860
...,...,...,...,...,...,...,...,...,...
592,Bandarbeyla,9.49,50.81,299.19,64,1,15.15,SO,1597071928
593,Chegdomyn,51.12,133.02,289.24,93,76,2.48,RU,1597071928
594,Tupiza,-21.45,-65.72,291.24,17,0,0.53,BO,1597071928
595,Gaoua,10.30,-3.25,303.82,58,82,4.24,BF,1597071727


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [23]:
gmaps.configure(api_key=g_key)

#Store latitude and longitude in locations
locations = df[["Lat", "Lng"]]

#Store Humidity in humidity
humidity = df["Humidity"]


In [24]:
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

#Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=5)

#Add layer
fig.add_layer(heat_layer)

#Display figure
fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [55]:
# Max_temp= df.loc[df["Max Temp"]>=70 & (df["Max Temp"]<80)]
# Max_temp

cities_weather=df.loc[(df["Max Temp"]>70) & (df["Wind Speed"]<10) & (df["Cloudiness"]==0)].dropna()
cities_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,Saldanha,-33.01,17.94,290.15,55,0,3.60,ZA,1597071861
10,Senador José Porfírio,-2.59,-51.95,304.72,47,0,1.00,BR,1597071861
11,New Norfolk,-42.78,147.06,274.82,89,0,1.30,AU,1597071861
16,Turukhansk,65.82,87.98,288.56,78,0,4.00,RU,1597071862
17,Tessalit,20.20,1.01,314.02,9,0,3.35,ML,1597071862
...,...,...,...,...,...,...,...,...,...
569,Ust'-Omchug,61.15,149.63,283.83,81,0,3.20,RU,1597071925
573,Quthing,-30.40,27.70,285.64,37,0,0.40,LS,1597071926
583,Semnan,35.57,53.40,307.63,16,0,1.40,IR,1597071927
588,Torres,-29.34,-49.73,297.81,49,0,3.87,BR,1597071928


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [61]:
hotel_df=pd.Dataframe(cities_weather)
hotel_df

AttributeError: module 'pandas' has no attribute 'Dataframe'

In [6]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'hotel_df' is not defined

In [ ]:
# Add marker layer ontop of heat map


# Display figure
